# Calculate the values to be used for normalization the entire training data

We need to looad it all into memory unfortunately

We calculate 1 mean per temperature and save it

In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
#%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
#%autoreload 2

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from joblib import dump, load

from src.data import make_dataset
from src.data import read_dataset
from src.data import util

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [4]:
from src.data.read_dataset import read_temperature

def make_temperatures(testing=False):
    num_cols = [x for x in range(22050)]
    save_cols = num_cols + ["label"]
    
    for i in range(2, 8):
        temperature = f"t0{i}"
        df = read_temperature(temperature)

        train_idx = df["training"] == 1  # get train data
        train_data = df.loc[train_idx]
        test_data = df.loc[~train_idx]

        train_data, val_data = train_test_split(train_data, test_size=0.2)
        
        scaler = StandardScaler()
        scaler.fit(train_data[num_cols])
        dump(scaler, f"../data/interim/scaler_{temperature}.pkl")
        
        # Save the data as compressed numpy arrays
        np.savez_compressed(f"../data/interim/all_wavs_{temperature}", 
                            train=train_data[save_cols].astype(int), 
                            val=val_data[save_cols].astype(int), 
                            test=test_data[save_cols].astype(int))

In [5]:
make_temperatures()

In [80]:
temp = np.load("../data/interim/all_wavs_t02.npz", allow_pickle=True)["val"]

In [81]:
temp = temp[:, :-1]

In [82]:
import joblib
scaler = joblib.load(f"../data/interim/scaler_t02.pkl")

In [123]:
example = scaler.transform(temp[0].reshape(1, -1))
example = example.reshape(-1)
example = example[:20]
example[:5] = -1
example[-5:] = +1

In [134]:
sr = 22050
roll_threshold = 0.1
roll_idx = int(roll_threshold*sr)
roll_idx = 2
roll_offset = np.random.randint(-roll_idx, +roll_idx)
left_extra = 0 if roll_offset <= 0 else roll_offset
right_extra = 0 if roll_offset >= 0 else -roll_offset
extra = ((left_extra),(right_extra))

print(roll_offset)
if roll_offset > 0:
    print(np.pad(example, extra)[:-roll_offset])
elif roll_offset < 0:
    print(np.pad(example, extra)[-roll_offset:])

-1
[-1. -1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.
  1.  0.]
